In [83]:
import random
import json
from pprint import pprint
import nltk
import re
import string
from nltk.tag.stanford import StanfordPOSTagger
from nltk.tag.stanford import StanfordNERTagger
from collections import defaultdict

### import Standford POS Tagger and Stanford NER Tagger

In [6]:
#to run StanfordPostTagger and NERTagger, 
#first download these two packages from http://nlp.stanford.edu/software/CRF-NER.shtml 
#I saved the downloaded files in lib/
post = StanfordPOSTagger('lib/stanford-postagger-2014-08-27/models/english-bidirectional-distsim.tagger', 
                         'lib/stanford-postagger-2014-08-27/stanford-postagger.jar', 'utf-8') # doctest: +SKIP
# post.tag('What is the airspeed of an unladen swallow ?'.split()) # doctest: +SKIP
nert = StanfordNERTagger('lib/stanford-ner-2014-08-27/classifiers/english.all.3class.distsim.crf.ser.gz',
                         'lib/stanford-ner-2014-08-27/stanford-ner.jar', 'utf-8')

### make a random sample of 2 and smaple of 5 from the json data as starting data.
Note: only run this code once. Currently commented out. 

In [7]:
# with open('movies-with-roles-summaries.json')as data_file:
#     data = json.load(data_file)

# print(type(data))

In [8]:
# data_sample_2 = random.sample((data.items()), 2)

In [9]:
# data_sample_5 = random.sample((data.items()), 5)

In [10]:
# with open('data_sample_2.json', 'w') as outfile:
#     json.dump(data_sample_2, outfile)

In [11]:
# with open('data_sample_5.json', 'w') as outfile:
#     json.dump(data_sample_5, outfile)

###Load in the sample of five

In [12]:
with open('data_sample_5.json') as sample:
# with open('movies-with-roles-summaries.json') as sample:
    sample = json.load(sample)

###More data cleaning

In [14]:
# get rid of ([[ in the wikipidia summaries
for i in range(len(sample)):
    summaries_wiki = sample[i][1]['summaries_wikipedia']
    if len(summaries_wiki) > 0:
        summaries_wiki = re.sub(r'\s\(\[\[', ' ', summaries_wiki[0])
        sample[i][1]['summaries_wikipedia'][0] = summaries_wiki

###Tagging

In [15]:
#pos tagging using nltk.pos_tag
#ner tagging using stanford ner tagger
def ie_preprocess(document, lower='false', stage="pos"):
#     sentences = nltk.sent_tokenize(document)
#     sentences = [nltk.word_tokenize(sent) for sent in sentences]
#     if lower == 'false':
#         for sent in sentences:
#             for i in range(len(sent)):
#                 if sent[i] != sent[i].lower():
#                     sent[i] = sent[i].lower()
#     sentences = [nltk.pos_tag(sent) for sent in sentences]
#     if stage == "pos":
#         return sentences
    if stage == "ner":
        return nert.tag(document.split())

In [16]:
# pos + ner tag summaries and roles
for i in range(len(sample)):
    summaries_imdb = sample[i][1]["summaries_imdb"]
    summaries_wiki = sample[i][1]['summaries_wikipedia']
    roles = sample[i][1]['roles']
    if len(summaries_imdb) > 0:
#         sample[i][1]["summaries_imdb_pos"] = ie_preprocess(summaries_imdb[0])
        sample[i][1]["summaries_imdb_ner"] = ie_preprocess(summaries_imdb[0], stage="ner")
    if len(summaries_wiki) > 0:
#         sample[i][1]["summaries_wikipedia_pos"] = ie_preprocess(summaries_wiki[0])
        sample[i][1]["summaries_wikipedia_ner"] = ie_preprocess(summaries_wiki[0], stage='ner')

###Search for a ner tagged name in the roles bag of words

####First, combine wiki summaries and imdb summaries into one string. 

In [37]:
# combine wiki_ner and imdb_ner
for i in range(len(sample)):
#     summaries_imdb_ner is indeed a list
    summaries_imdb_ner = sample[i][1]["summaries_imdb_ner"]
    if 'summaries_wikipedia_ner' in sample[i][1]:
        summaries_wiki_ner = sample[i][1]['summaries_wikipedia_ner']
    else: summaries_wiki_ner = []
    summaries_imdb_ner.extend(summaries_wiki_ner)
    sample[i][1]['summaries_combined_ner'] = summaries_imdb_ner

####Run chuncker to lift out tagged names from summaries

In [125]:
# creating a new dict called sum_and_char to store summaries and corresponding characters
sum_and_char = {}
for i in range(len(sample)):
#     making a bag of words for roles
    roles_bag = []
    roles = sample[i][1]['roles']
    for j in range(len(roles)):
        roles_bag.append(roles[j]['role'])
    str = " ".join(roles_bag)
#     split roles on space, /, ' or "
    roles_bag = re.split(r' |/|\'|\"', str)
#     sorting entity_chuncking results into summaries and characters
    result = entity_chunker(sample[i][1]['summaries_combined_ner'])
    movie_name = sample[i][0]
    if len(result) <= 1:
        char_data = {}
    else:
        char_data = set(result[1:])
    sum_and_char[movie_name] = {"sum": result[0], \
                                "char_raw": char_data,\
                                "roles_bag": set(roles_bag),\
                                "char_info" : {}}
# pprint(sum_and_char)

{'angelica.2015': {'char_info': {},
                   'char_raw': {'Constance', 'Mitchell', 'Joseph'},
                   'roles_bag': {'Angelica',
                                 'Anne',
                                 'Barton',
                                 'Beggar',
                                 'Bookseller',
                                 'Boy',
                                 'Constance',
                                 'Dr.',
                                 'Girl',
                                 'Joseph',
                                 'Lady',
                                 'Medical',
                                 'Midwife',
                                 'Montague',
                                 'Mr.',
                                 'Nora',
                                 'Older',
                                 'Park',
                                 'Pendleton',
                                 'Scientist',
                                 'Sno

####Check characters against roles for filtering

In [126]:
# if any one word in a character name appears in the roles_bag, then this name is appended to char_filter
# print("*************************")
# print("filtered character list")
# print("*************************")

for key, value in sum_and_char.items():
    sum_and_char[key]['char_filtered'] = []
    for char in value['char_raw']:
        flag = False
        char_split = char.split()
        for elem in char_split:
            if elem in value['roles_bag']:
                flag = True
        if flag == True:
            sum_and_char[key]['char_filtered'].append(char)
#     print(sum_and_char[key]['char_filtered'])
            
# compare with the raw list of characters before filtering
# "dorcy" is not captured from richard palmer grant
# TODO: capture capitalized word right after the character name and make it part of the character name.
# print("*************************")
# print("raw character list")
# print("*************************")

# for value in sum_and_char.values():
# #     pprint(values):
#     for char in value['char_raw']:
#         print(char)
        

*************************
filtered character list
*************************
[]
[]
["Scrapper Thornhill Pat O'Brien", 'Biddle', 'Biddle John Eldredge', 'Kit', 'Dick Powell', 'Dick', 'Fitts']
['Abraham Lincoln', 'Alison Gardner', 'Elliot Richards', 'Nicole Delarusso', 'Elliot', 'Alison', 'Bob']
['Constance', 'Joseph']
*************************
raw character list
*************************
Ishmael Bernal
Richard Palmer Grant
Scrapper Thornhill Pat O'Brien
Biddle
Biddle John Eldredge
Kit
Dick Powell
Dick
Fitts
Abraham Lincoln
Alison Gardner
Satan
Elliot Richards
Nicole Delarusso
Elliot
Alison
Bob
Constance
Mitchell
Joseph


###Establish the link between filtered character names from summaries and their counter parts in the roles list

In [146]:
# TODO: add algorithm for tie-breaking

# link char_filter with their names in the role list.
''' char_info in sum_and_char use names in the role list as keys, and contains role gender, char names
found in summaries, and roles found in summaries
'''
for key, value in (sum_and_char.items()):
    for i in range(len(sample)):
#     locate correct movie
        if sample[i][0] == key:
#             for each filted char name
            for name in value['char_filtered']:
                max_count = 0
                max_role = None
                max_gen = None
                name_split = name.split()
#                 for each role
                for j in range(len(sample[i][1]['roles'])):
                    count = 0
#                   for each word in char name
#                   increment count by 1 every time a word in the char name appears in a role
                    for k in range(len(name_split)):
                        if name_split[k] in sample[i][1]['roles'][j]['role_bag']:
                            count += 1
#                             print(name_split[k])
#                             print(name)
#                             print(count)
#                   select the role with the most counts!
                    if count > max_count:
                        max_count = count
                        max_role = sample[i][1]['roles'][j]['role']
                        max_gen = sample[i][1]['roles'][j]['gender']
                if max_count < 1:
#                     print("error")
                else:
#                     print(name)
#                     print(max_count)
#                     print(max_role)
#                     print(max_gen)
#                     print()
                    if max_role not in value['char_info']:
                        value['char_info'][max_role] = {'gender':max_gen, 'roles_found_in_sums':[], 'names_found_in_sums':[name]}
                    else:
                        value['char_info'][max_role]['names_found_in_sums'].append(name)
                        
                        
                    
                    
                    

Scrapper Thornhill Pat O'Brien
1
'Scrapper' Thornhhill
M

Biddle
1
Lieut. Biddle
M

Biddle John Eldredge
1
Lieut. Biddle
M

Kit
1
'Kit' Fitts
F

Dick Powell
1
Dick 'Canary' Dorcy
M

Dick
1
Dick 'Canary' Dorcy
M

Fitts
1
'Kit' Fitts
F

Abraham Lincoln
1
Bob/Roberto/Beach Jock/Sportscaster/Lincoln Aide
M

Alison Gardner
1
Alison/Nicole
F

Elliot Richards
1
Elliot's Cellmate
M

Nicole Delarusso
1
Alison/Nicole
F

Elliot
1
Elliot's Cellmate
M

Alison
1
Alison/Nicole
F

Bob
1
Bob/Roberto/Beach Jock/Sportscaster/Lincoln Aide
M

Constance
1
Constance
F

Joseph
1
Dr. Joseph Barton
M

